In [68]:
#Import the dependencies
from bs4 import BeautifulSoup
# import pandas as pd
import requests
# import urllib.request
import time
import pandas as pd
import numpy as np

# Creating custom scraping function for Poynter

In [10]:
#Create lists to store the scraped data
source_list = []
date_list = []
country_list = []
label_list = []
title_list = []
true_explanation_list = []

In [136]:
#Create a function to scrape the site
def scrape_poynter(start_page, end_page):
    
    #initialising the empty list of information to collect
#     source_list = []
#     date_list = []
#     country_list = []
#     label_list = []
#     title_list = []
#     true_explanation_list = []

    #Looping over the pages
    for page_num in range(start_page-1, end_page):
        
        URL = 'https://www.poynter.org/ifcn-covid-19-misinformation/page/' + str(page_num+1) #append the page number to complete the URL
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
        
        #Add delay to request for explanation page, if required
#         time.sleep(2)

        webpage = requests.get(URL, headers=headers)  #Make a request to the website

        #Print statement for iteration
        print('-'*50)
        print(f'Extracting information from page number {page_num+1}')
        
        webpage.raise_for_status() #Raise error if page unavailable
        
        soup = BeautifulSoup(webpage.content, "html.parser") #Parse the text from the website

        # Get source, date and country
        source_date_country_list = soup.find_all('p',attrs={'class':'entry-content__text'})

        for id_, item in enumerate(source_date_country_list):
            if id_%2 == 0:
                source_list.append(item.text.replace('Fact-Checked by: ', ''))
            else:
                date_list.append(item.text[:10])
                country_list.append(item.text[13:])

        #Get labels and title
        source_title_list = soup.find_all('h2', attrs={'class':'entry-title'})

        for item in source_title_list:
            label = item.find('span').text
            label_list.append(label.replace(':', ''))

            title = item.text.replace(label, '')
            title_list.append(title.strip())

        #Get link to explanation (for true)
        explanation_pages = soup.find_all('a', attrs={'class':'button entry-content__button entry-content__button--smaller'})

        for id_, item in enumerate(explanation_pages):
            article_url = item['href']

            #Add delay to request for explanation page
#             time.sleep(2)

            article_requests = requests.get(article_url, headers=headers)

            #Add print statement for iteration
            print(f'Extracting true explanation for item {id_+1} from page number {page_num+1}')

            article_requests.raise_for_status()

            article_soup = BeautifulSoup(article_requests.content, "html.parser") #Parse the text from the website

            true_explanation = article_soup.find('p', attrs={'class':'entry-content__text entry-content__text--explanation'}).text. \
                 replace('Explanation: ', '')

            true_explanation_list.append(true_explanation)

#     return source_list, date_list, country_list, label_list, title_list, true_explanation_list

In [11]:
scrape_poynter(500)

--------------------------------------------------
Extracting information from page number 1
Extracting true explanation for item 1 from page number 1
Extracting true explanation for item 2 from page number 1
Extracting true explanation for item 3 from page number 1
Extracting true explanation for item 4 from page number 1
Extracting true explanation for item 5 from page number 1
Extracting true explanation for item 6 from page number 1
Extracting true explanation for item 7 from page number 1
Extracting true explanation for item 8 from page number 1
Extracting true explanation for item 9 from page number 1
Extracting true explanation for item 10 from page number 1
Extracting true explanation for item 11 from page number 1
Extracting true explanation for item 12 from page number 1
Extracting true explanation for item 13 from page number 1
Extracting true explanation for item 14 from page number 1
Extracting true explanation for item 15 from page number 1
-------------------------------

Extracting true explanation for item 7 from page number 9
Extracting true explanation for item 8 from page number 9
Extracting true explanation for item 9 from page number 9
Extracting true explanation for item 10 from page number 9
Extracting true explanation for item 11 from page number 9
Extracting true explanation for item 12 from page number 9
Extracting true explanation for item 13 from page number 9
Extracting true explanation for item 14 from page number 9
Extracting true explanation for item 15 from page number 9
--------------------------------------------------
Extracting information from page number 10
Extracting true explanation for item 1 from page number 10
Extracting true explanation for item 2 from page number 10
Extracting true explanation for item 3 from page number 10
Extracting true explanation for item 4 from page number 10
Extracting true explanation for item 5 from page number 10
Extracting true explanation for item 6 from page number 10
Extracting true explanat

Extracting true explanation for item 13 from page number 17
Extracting true explanation for item 14 from page number 17
Extracting true explanation for item 15 from page number 17
--------------------------------------------------
Extracting information from page number 18
Extracting true explanation for item 1 from page number 18
Extracting true explanation for item 2 from page number 18
Extracting true explanation for item 3 from page number 18
Extracting true explanation for item 4 from page number 18
Extracting true explanation for item 5 from page number 18
Extracting true explanation for item 6 from page number 18
Extracting true explanation for item 7 from page number 18
Extracting true explanation for item 8 from page number 18
Extracting true explanation for item 9 from page number 18
Extracting true explanation for item 10 from page number 18
Extracting true explanation for item 11 from page number 18
Extracting true explanation for item 12 from page number 18
Extracting true

Extracting true explanation for item 2 from page number 26
Extracting true explanation for item 3 from page number 26
Extracting true explanation for item 4 from page number 26
Extracting true explanation for item 5 from page number 26
Extracting true explanation for item 6 from page number 26
Extracting true explanation for item 7 from page number 26
Extracting true explanation for item 8 from page number 26
Extracting true explanation for item 9 from page number 26
Extracting true explanation for item 10 from page number 26
Extracting true explanation for item 11 from page number 26
Extracting true explanation for item 12 from page number 26
Extracting true explanation for item 13 from page number 26
Extracting true explanation for item 14 from page number 26
Extracting true explanation for item 15 from page number 26
--------------------------------------------------
Extracting information from page number 27
Extracting true explanation for item 1 from page number 27
Extracting true

Extracting true explanation for item 8 from page number 34
Extracting true explanation for item 9 from page number 34
Extracting true explanation for item 10 from page number 34
Extracting true explanation for item 11 from page number 34
Extracting true explanation for item 12 from page number 34
Extracting true explanation for item 13 from page number 34
Extracting true explanation for item 14 from page number 34
Extracting true explanation for item 15 from page number 34
--------------------------------------------------
Extracting information from page number 35
Extracting true explanation for item 1 from page number 35
Extracting true explanation for item 2 from page number 35
Extracting true explanation for item 3 from page number 35
Extracting true explanation for item 4 from page number 35
Extracting true explanation for item 5 from page number 35
Extracting true explanation for item 6 from page number 35
Extracting true explanation for item 7 from page number 35
Extracting true

...

Extracting true explanation for item 5 from page number 464
Extracting true explanation for item 6 from page number 464
Extracting true explanation for item 7 from page number 464
Extracting true explanation for item 8 from page number 464
Extracting true explanation for item 9 from page number 464
Extracting true explanation for item 10 from page number 464
Extracting true explanation for item 11 from page number 464
Extracting true explanation for item 12 from page number 464
Extracting true explanation for item 13 from page number 464
Extracting true explanation for item 14 from page number 464
Extracting true explanation for item 15 from page number 464
--------------------------------------------------
Extracting information from page number 465
Extracting true explanation for item 1 from page number 465
Extracting true explanation for item 2 from page number 465
Extracting true explanation for item 3 from page number 465
Extracting true explanation for item 4 from page number 465

Extracting true explanation for item 9 from page number 472
Extracting true explanation for item 10 from page number 472
Extracting true explanation for item 11 from page number 472
Extracting true explanation for item 12 from page number 472
Extracting true explanation for item 13 from page number 472
Extracting true explanation for item 14 from page number 472
Extracting true explanation for item 15 from page number 472
--------------------------------------------------
Extracting information from page number 473
Extracting true explanation for item 1 from page number 473
Extracting true explanation for item 2 from page number 473
Extracting true explanation for item 3 from page number 473
Extracting true explanation for item 4 from page number 473
Extracting true explanation for item 5 from page number 473
Extracting true explanation for item 6 from page number 473
Extracting true explanation for item 7 from page number 473
Extracting true explanation for item 8 from page number 473

Extracting true explanation for item 13 from page number 480
Extracting true explanation for item 14 from page number 480
Extracting true explanation for item 15 from page number 480
--------------------------------------------------
Extracting information from page number 481
Extracting true explanation for item 1 from page number 481
Extracting true explanation for item 2 from page number 481
Extracting true explanation for item 3 from page number 481
Extracting true explanation for item 4 from page number 481
Extracting true explanation for item 5 from page number 481
Extracting true explanation for item 6 from page number 481
Extracting true explanation for item 7 from page number 481
Extracting true explanation for item 8 from page number 481
Extracting true explanation for item 9 from page number 481
Extracting true explanation for item 10 from page number 481
Extracting true explanation for item 11 from page number 481
Extracting true explanation for item 12 from page number 481

Extracting true explanation for item 1 from page number 489
Extracting true explanation for item 2 from page number 489
Extracting true explanation for item 3 from page number 489
Extracting true explanation for item 4 from page number 489
Extracting true explanation for item 5 from page number 489
Extracting true explanation for item 6 from page number 489
Extracting true explanation for item 7 from page number 489
Extracting true explanation for item 8 from page number 489
Extracting true explanation for item 9 from page number 489
Extracting true explanation for item 10 from page number 489
Extracting true explanation for item 11 from page number 489
Extracting true explanation for item 12 from page number 489
Extracting true explanation for item 13 from page number 489
Extracting true explanation for item 14 from page number 489
Extracting true explanation for item 15 from page number 489
--------------------------------------------------
Extracting information from page number 490

Extracting true explanation for item 5 from page number 497
Extracting true explanation for item 6 from page number 497
Extracting true explanation for item 7 from page number 497
Extracting true explanation for item 8 from page number 497
Extracting true explanation for item 9 from page number 497
Extracting true explanation for item 10 from page number 497
Extracting true explanation for item 11 from page number 497
Extracting true explanation for item 12 from page number 497
Extracting true explanation for item 13 from page number 497
Extracting true explanation for item 14 from page number 497
Extracting true explanation for item 15 from page number 497
--------------------------------------------------
Extracting information from page number 498
Extracting true explanation for item 1 from page number 498
Extracting true explanation for item 2 from page number 498
Extracting true explanation for item 3 from page number 498
Extracting true explanation for item 4 from page number 498

In [91]:
source_list, date_list, country_list, label_list, title_list, true_explanation_list = scrape_poynter(500)

--------------------------------------------------
Extracting information from page number 1
Extracting true explanation for item 1 from page number 1
Extracting true explanation for item 2 from page number 1
Extracting true explanation for item 3 from page number 1
Extracting true explanation for item 4 from page number 1
Extracting true explanation for item 5 from page number 1
Extracting true explanation for item 6 from page number 1
Extracting true explanation for item 7 from page number 1
Extracting true explanation for item 8 from page number 1
Extracting true explanation for item 9 from page number 1
Extracting true explanation for item 10 from page number 1
Extracting true explanation for item 11 from page number 1
Extracting true explanation for item 12 from page number 1
Extracting true explanation for item 13 from page number 1
Extracting true explanation for item 14 from page number 1
Extracting true explanation for item 15 from page number 1
-------------------------------

Extracting true explanation for item 7 from page number 9
Extracting true explanation for item 8 from page number 9
Extracting true explanation for item 9 from page number 9
Extracting true explanation for item 10 from page number 9
Extracting true explanation for item 11 from page number 9
Extracting true explanation for item 12 from page number 9
Extracting true explanation for item 13 from page number 9
Extracting true explanation for item 14 from page number 9
Extracting true explanation for item 15 from page number 9
--------------------------------------------------
Extracting information from page number 10
Extracting true explanation for item 1 from page number 10
Extracting true explanation for item 2 from page number 10
Extracting true explanation for item 3 from page number 10
Extracting true explanation for item 4 from page number 10
Extracting true explanation for item 5 from page number 10
Extracting true explanation for item 6 from page number 10
Extracting true explanat

Extracting true explanation for item 13 from page number 17
Extracting true explanation for item 14 from page number 17
Extracting true explanation for item 15 from page number 17
--------------------------------------------------
Extracting information from page number 18
Extracting true explanation for item 1 from page number 18
Extracting true explanation for item 2 from page number 18
Extracting true explanation for item 3 from page number 18
Extracting true explanation for item 4 from page number 18
Extracting true explanation for item 5 from page number 18
Extracting true explanation for item 6 from page number 18
Extracting true explanation for item 7 from page number 18
Extracting true explanation for item 8 from page number 18
Extracting true explanation for item 9 from page number 18
Extracting true explanation for item 10 from page number 18
Extracting true explanation for item 11 from page number 18
Extracting true explanation for item 12 from page number 18
Extracting true

Extracting true explanation for item 2 from page number 26
Extracting true explanation for item 3 from page number 26
Extracting true explanation for item 4 from page number 26
Extracting true explanation for item 5 from page number 26
Extracting true explanation for item 6 from page number 26
Extracting true explanation for item 7 from page number 26
Extracting true explanation for item 8 from page number 26
Extracting true explanation for item 9 from page number 26
Extracting true explanation for item 10 from page number 26
Extracting true explanation for item 11 from page number 26
Extracting true explanation for item 12 from page number 26
Extracting true explanation for item 13 from page number 26
Extracting true explanation for item 14 from page number 26
Extracting true explanation for item 15 from page number 26
--------------------------------------------------
Extracting information from page number 27
Extracting true explanation for item 1 from page number 27
Extracting true

Extracting true explanation for item 8 from page number 34
Extracting true explanation for item 9 from page number 34
Extracting true explanation for item 10 from page number 34
Extracting true explanation for item 11 from page number 34
Extracting true explanation for item 12 from page number 34
Extracting true explanation for item 13 from page number 34
Extracting true explanation for item 14 from page number 34
Extracting true explanation for item 15 from page number 34
--------------------------------------------------
Extracting information from page number 35
Extracting true explanation for item 1 from page number 35
Extracting true explanation for item 2 from page number 35
Extracting true explanation for item 3 from page number 35
Extracting true explanation for item 4 from page number 35
Extracting true explanation for item 5 from page number 35
Extracting true explanation for item 6 from page number 35
Extracting true explanation for item 7 from page number 35
Extracting true

...

Extracting true explanation for item 9 from page number 274
Extracting true explanation for item 10 from page number 274
Extracting true explanation for item 11 from page number 274
Extracting true explanation for item 12 from page number 274
Extracting true explanation for item 13 from page number 274
Extracting true explanation for item 14 from page number 274
Extracting true explanation for item 15 from page number 274
--------------------------------------------------
Extracting information from page number 275
Extracting true explanation for item 1 from page number 275
Extracting true explanation for item 2 from page number 275
Extracting true explanation for item 3 from page number 275
Extracting true explanation for item 4 from page number 275
Extracting true explanation for item 5 from page number 275
Extracting true explanation for item 6 from page number 275
Extracting true explanation for item 7 from page number 275
Extracting true explanation for item 8 from page number 275

Extracting true explanation for item 13 from page number 282
Extracting true explanation for item 14 from page number 282
Extracting true explanation for item 15 from page number 282
--------------------------------------------------
Extracting information from page number 283
Extracting true explanation for item 1 from page number 283
Extracting true explanation for item 2 from page number 283
Extracting true explanation for item 3 from page number 283
Extracting true explanation for item 4 from page number 283
Extracting true explanation for item 5 from page number 283
Extracting true explanation for item 6 from page number 283
Extracting true explanation for item 7 from page number 283
Extracting true explanation for item 8 from page number 283
Extracting true explanation for item 9 from page number 283
Extracting true explanation for item 10 from page number 283
Extracting true explanation for item 11 from page number 283
Extracting true explanation for item 12 from page number 283

Extracting true explanation for item 1 from page number 291
Extracting true explanation for item 2 from page number 291
Extracting true explanation for item 3 from page number 291
Extracting true explanation for item 4 from page number 291
Extracting true explanation for item 5 from page number 291
Extracting true explanation for item 6 from page number 291
Extracting true explanation for item 7 from page number 291
Extracting true explanation for item 8 from page number 291
Extracting true explanation for item 9 from page number 291
Extracting true explanation for item 10 from page number 291
Extracting true explanation for item 11 from page number 291
Extracting true explanation for item 12 from page number 291
Extracting true explanation for item 13 from page number 291
Extracting true explanation for item 14 from page number 291
Extracting true explanation for item 15 from page number 291
--------------------------------------------------
Extracting information from page number 292

Extracting true explanation for item 5 from page number 299
Extracting true explanation for item 6 from page number 299
Extracting true explanation for item 7 from page number 299
Extracting true explanation for item 8 from page number 299
Extracting true explanation for item 9 from page number 299
Extracting true explanation for item 10 from page number 299
Extracting true explanation for item 11 from page number 299
Extracting true explanation for item 12 from page number 299
Extracting true explanation for item 13 from page number 299
Extracting true explanation for item 14 from page number 299
Extracting true explanation for item 15 from page number 299
--------------------------------------------------
Extracting information from page number 300
Extracting true explanation for item 1 from page number 300
Extracting true explanation for item 2 from page number 300
Extracting true explanation for item 3 from page number 300
Extracting true explanation for item 4 from page number 300

KeyboardInterrupt: 

# Create the DataFrame from the scraped information

In [12]:
assert len(title_list) == len(true_explanation_list)

In [13]:
len(title_list)

7500

In [17]:
df = pd.DataFrame(dict([('source_list', source_list), ('date_list', date_list), ('country_list', country_list),
      ('label_list',label_list),('title_list',title_list)]))

df.head()

,source_list,date_list,country_list,label_list,title_list
0,Maldita.es,2021/08/13,Spain,FALSO,There is a call for free vaccination in the Ci...
1,Maldita.es,2021/08/12,Spain,FALSO,Video of a girl being forcibly vaccinated agai...
2,Maldita.es,2021/08/12,"Spain, United States",FALSO,The US CDC has admitted an error in the PCR te...
3,Maldita.es,2021/08/11,Spain,FALSO,Children are 50 times more likely to die from ...
4,Maldita.es,2021/08/10,Spain,FALSO,Those vaccinated against the COVID-19 cannot d...


In [22]:
true_df = pd.DataFrame(dict([('true_explanation_list',true_explanation_list)]))

true_df.head()

,true_explanation_list
0,It is required to go by appointment.
1,The girl has uploaded another video in which s...
2,The CDC does not question the reliability of P...
3,Yeadon does not provide any proof of his claim...
4,"In Spain, according to the Ministry of Health,..."


In [45]:
df['label_list_cleaned'] = df['label_list'].apply(lambda x: x.lower())

df.head()

,source_list,date_list,country_list,label_list,title_list,label_list_cleaned
0,Maldita.es,2021/08/13,Spain,FALSO,There is a call for free vaccination in the Ci...,falso
1,Maldita.es,2021/08/12,Spain,FALSO,Video of a girl being forcibly vaccinated agai...,falso
2,Maldita.es,2021/08/12,"Spain, United States",FALSO,The US CDC has admitted an error in the PCR te...,falso
3,Maldita.es,2021/08/11,Spain,FALSO,Children are 50 times more likely to die from ...,falso
4,Maldita.es,2021/08/10,Spain,FALSO,Those vaccinated against the COVID-19 cannot d...,falso


In [135]:
#Getting unique values for Poynter's conclusion
df['label_list_cleaned'].unique()

array(['falso', 'false', 'misleading', 'no evidence', 'partly false',
       'f aux', 'party false', 'fake', 'missing context', 'partl false',
       'manipulation', 'explanatory', 'conspiracy', 'fałsz', 'enganoso',
       'faux', 'half truth', 'misleasing', 'out of context',
       'exaggerated', 'satire', 'altered image', 'unproven', 'true',
       'missleading', 'misleanding', 'mix', 'mostly false', 'exagerated',
       'misleading/false', 'mainly false', 'unsustainable', 'inaccurate',
       'partially false', 'missiing context', 'manipulated',
       'mainly correct', 'mostly true', 'partially true',
       'false headline', 'sarcastic', 'explicative'], dtype=object)

In [89]:
def label_mapping(x):
    '''Function to map unique labels to true/false values'''
    
    mapping = {'falso': 1,
                 'false': 1,
                 'misleading': 1,
                 'no evidence': 1,
                 'partly false': 1,
                 'f aux': 1,
                 'party false': 1,
                 'fake': 1,
                 'missing context': 1,
                 'partl false': 1,
                 'manipulation': 1,
                 'explanatory': 1,
                 'conspiracy': 1,
                 'fałsz': 1,
                 'enganoso': 1,
                 'faux': 1,
                 'half truth': 1,
                 '': np.nan,
                 'misleasing': 1,
                 'out of context': 1,
                 'exaggerated': 1,
                 'satire': 1,
                 'altered image': 1,
                 'unproven': 1,
                 'true': 0,
                 'missleading': 1,
                 'misleanding': 1,
                 'mix': 1,
                 'mostly false': 1,
                 'exagerated': 1,
                 'misleading/false': 1,
                 'mainly false': 1,
                 'unsustainable': 1,
                 'inaccurate': 1,
                 'partially false': 1,
                 'missiing context': 1,
                 'manipulated': 1,
                 'mainly correct': 0,
                 'mostly true': 0,
                 'partially true': 0,
                 'false headline': 1,
                 'sarcastic': 1,
                 'explicative': 1}
    
    if x in mapping.keys():
        return mapping[x]
    
    return x

In [90]:
df['label_list_transformed'] = df['label_list_cleaned'].apply(label_mapping)

In [91]:
df.head()

,source_list,date_list,country_list,label_list,title_list,label_list_cleaned,label_list_transformed
0,Maldita.es,2021/08/13,Spain,FALSO,There is a call for free vaccination in the Ci...,falso,1.0
1,Maldita.es,2021/08/12,Spain,FALSO,Video of a girl being forcibly vaccinated agai...,falso,1.0
2,Maldita.es,2021/08/12,"Spain, United States",FALSO,The US CDC has admitted an error in the PCR te...,falso,1.0
3,Maldita.es,2021/08/11,Spain,FALSO,Children are 50 times more likely to die from ...,falso,1.0
4,Maldita.es,2021/08/10,Spain,FALSO,Those vaccinated against the COVID-19 cannot d...,falso,1.0


In [110]:
#Removing empty labels and empty titles from df

df = df.drop(index=df[df['label_list_transformed'].isna()].index, axis=0)
df = df.drop(index=df[df['title_list']==''].index, axis=0)

df.shape

(7450, 7)

In [111]:
#Removing empty titles from true_df

true_df = true_df.drop(index=true_df[true_df['true_explanation_list']==''].index, axis=0)

true_df.shape

(7447, 1)

In [126]:
true_df = true_df.rename(columns={'true_explanation_list':'title_list'})
true_df['label_list_transformed'] = 0

true_df.head()

,title_list,label_list_transformed
0,It is required to go by appointment.,0
1,The girl has uploaded another video in which s...,0
2,The CDC does not question the reliability of P...,0
3,Yeadon does not provide any proof of his claim...,0
4,"In Spain, according to the Ministry of Health,...",0


In [129]:
final_df = pd.concat([df[['title_list','label_list_transformed']],true_df],axis=0)

final_df.head()

,title_list,label_list_transformed
0,There is a call for free vaccination in the Ci...,1.0
1,Video of a girl being forcibly vaccinated agai...,1.0
2,The US CDC has admitted an error in the PCR te...,1.0
3,Children are 50 times more likely to die from ...,1.0
4,Those vaccinated against the COVID-19 cannot d...,1.0


In [130]:
assert final_df.shape[0] == (true_df.shape[0]+df.shape[0])

## Save to local

In [31]:
import os

In [36]:
df.to_csv(f"{os.path.join(os.path.dirname(os.getcwd()),'raw_data', 'poynter_fake.csv')}")

In [37]:
true_df.to_csv(f"{os.path.join(os.path.dirname(os.getcwd()),'raw_data', 'poynter_true.csv')}")

In [131]:
final_df.to_csv(f"{os.path.join(os.path.dirname(os.getcwd()),'raw_data', 'poynter_final_condensed.csv')}")

# Custom class Found Online (not tested yet)

In [4]:
import scrapy


class PoynterFakenewsSpider(scrapy.Spider):
    name = 'Poynter_FakeNews'
    allowed_domains = ['poynter.org']
    start_urls = ['https://www.poynter.org/ifcn-covid-19-misinformation//']

    custom_settings={ 'FEED_URI': "crawlPoynter_%(time)s.csv", 'FEED_FORMAT': 'csv'} 

    def parse(self, response):

        for article in response.xpath('//article'):
            Title = article.xpath('.//h2[@class="entry-title"]/a/text()').get()
            Date = article.xpath('.//p[@class="entry-content__text"]/strong/text()').get()
            ReadMore_link = article.xpath('.//a[@class="button entry-content__button entry-content__button--smaller"]/@href').get()

            yield scrapy.Request(
                url=response.urljoin(ReadMore_link), 
                callback=self.parse_article_details,
                cb_kwargs={
                    'article_title': Title,
                    'article_date': Date,
                }
            )
        next_page = response.xpath('//a[@class="next page-numbers"]/@href').extract_first()
        if next_page: 
            yield scrapy.Request(response.urljoin(next_page), callback=self.parse)

    def parse_article_details(self, response, article_title, article_date):
        Source = response.xpath('//p[@class="entry-content__text entry-content__text--smaller"]/text()').extract_first()
        scraped_info = {
            'page':response.url,
            'Title': article_title, 
            'Date': article_date,
            'Source': Source,
        }
        yield scraped_info

ModuleNotFoundError: No module named 'scrapy'